In [2]:
import pandas as pd
import os
import re
import xlsxwriter 
from openpyxl import load_workbook

In [16]:
bsFolder = "Nov2020_ConversionResult"
outputFolder = "Nov2020_ManipulationResult"
os.listdir(bsFolder)

['01_11 3207971832_2020.xlsx',
 '02_11 3207971832_2020.xlsx',
 '03_11 3207971832_2020.xlsx',
 '04_11 3207971832_2020.xlsx',
 '05_11 3207971832_2020.xlsx',
 '06_11 3207971832_2020.xlsx',
 '07_11 3207971832_2020.xlsx',
 '08_11 3207971832_2020.xlsx',
 '09_11 3207971832_2020.xlsx',
 '10_11 3207971832_2020.xlsx',
 '11_11 3207971832_2020.xlsx',
 '12_11 3207971832_2020.xlsx',
 '13_11 3207971832_2020.xlsx',
 '14_11 3207971832_2020.xlsx',
 '15_11 3207971832_2020.xlsx',
 '16_11 3207971832_2020.xlsx',
 '17_11 3207971832_2020.xlsx',
 '18_11 3207971832_2020.xlsx',
 '19_11 3207971832_2020.xlsx',
 '20_11 3207971832_2020.xlsx',
 '21_11 3207971832_2020.xlsx',
 '22_11 3207971832_2020.xlsx',
 '23_11 3207971832_2020.xlsx',
 '24_11 3207971832_2020.xlsx',
 '25_11 3207971832_2020.xlsx',
 '26_11 3207971832_2020.xlsx',
 '27_11 3207971832_2020.xlsx',
 '28_11 3207971832_2020.xlsx',
 '29_11 3207971832_2020.xlsx',
 '30_11 3207971832_2020.xlsx']

In [17]:
bsfile = "03_11 3207971832_2020.xlsx"
df = pd.read_excel(bsFolder + '/' + bsfile, skiprows = 9)
df.columns

Index(['Trn. Date', 'Reference 1', 'Debit Amount', 'Credit Amount', 'Balance'], dtype='object')

In [18]:
dfSummary = pd.read_excel(bsFolder + '/' + bsfile, nrows = 7)
dfSummary.iloc[-1,:]

Nombor Akaun / Account Number:    Jumlah Kredit / Total Credits:
3207971832                                         11,208,957.25
Unnamed: 2                                                   NaN
Unnamed: 3                                                   NaN
Unnamed: 4                                                   NaN
Name: 6, dtype: object

In [19]:
balanceBFIndex = [no for no, i in enumerate(df['Reference 1'].values) if re.search("Balance B/F", i)]

descType = ["INSTANT TRSF", "DEP-CASH", "HANDLING CHRG",
            "MISC DR", "DR-ECP", "DEP-ECP", "RTN-ITEM",
            "GIRO PYMT", "TRSF FUND", "TSFR FUND"]
#index:addDesc
addDescDict = {}
for index in balanceBFIndex:
    ref = df['Reference 1'].values[index + 1]
    if re.search("(.+?)(%s)"%'|'.join([re.escape(i) for i in descType]), ref):
        #row before c/f
        addDescDict[index - 2] = re.search("(.+)(%s)"%'|'.join([re.escape(i) for i in descType]), ref).group(1)

refList = []
for no, ref in enumerate(df['Reference 1'].values):
    if no in addDescDict.keys():
        refList.append(ref + ' ' + addDescDict[no])
    else:
        refList.append(ref)

df['Reference 2'] = refList

# df['NotSameReference'] = df['Reference 1'] != df['Reference 2']

df.columns

Index(['Trn. Date', 'Reference 1', 'Debit Amount', 'Credit Amount', 'Balance',
       'Reference 2'],
      dtype='object')

In [10]:
outputFolder + '/' + bsfile

'Nov2020_ManipulationResult/01_11 3207971832_2020.xlsx'

In [20]:
import xlsxwriter 
  
excelFile = outputFolder + '/' + bsfile
print(excelFile)
workbook = xlsxwriter.Workbook(excelFile)
worksheet = workbook.add_worksheet() 
workbook.close()

from openpyxl import load_workbook
#write pandas 
writer = pd.ExcelWriter(excelFile, engine='openpyxl')
writer.book = load_workbook(excelFile)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
dfSummary.to_excel(writer, sheet_name='Sheet1', header = True, index = False)
df[["Trn. Date", "Reference 1", "Reference 2", 'Debit Amount', 'Credit Amount', 'Balance']]\
.to_excel(writer, sheet_name='Sheet1', startrow = dfSummary.shape[0] + 2, header = True, index = False)
writer.save()

Nov2020_ManipulationResult/03_11 3207971832_2020.xlsx


### For loop

In [5]:
# bsFolder = "Nov2020_ConversionResult"
bsFolder = "Dec2020_ConversionResult"
# outputFolder = "Nov2020_ManipulationResult"
outputFolder = "Dec2020_ManipulationResult"

for bsfile in os.listdir(bsFolder):
    df = pd.read_excel(bsFolder + '/' + bsfile, skiprows = 9)
    dfSummary = pd.read_excel(bsFolder + '/' + bsfile, nrows = 7)
    
    balanceBFIndex = [no for no, i in enumerate(df['Reference 1'].values) if re.search("Balance B/F", i)]
    descType = ["INSTANT TRSF", "DEP-CASH", "HANDLING CHRG",
                "MISC DR", "DR-ECP", "DEP-ECP", "RTN-ITEM",
                "GIRO PYMT", "TRSF FUND"]
    #index:addDesc
    addDescDict = {}
    removeDescDict = {}
    for index in balanceBFIndex:
        ref = df['Reference 1'].values[index + 1]
        if re.search("(.+?)(%s)"%'|'.join([re.escape(i) for i in descType]), ref):
            #row before c/f
            addDescDict[index - 2] = re.search("(.+)(%s)"%'|'.join([re.escape(i) for i in descType]), ref).group(1)
            removeDesc = re.search("(.+)(%s)"%'|'.join([re.escape(i) for i in descType]), ref).group(1)
            removeDescDict[index + 1] = re.sub(re.escape(removeDesc), '', ref)

    refList = []
    for no, ref in enumerate(df['Reference 1'].values):
        if no in addDescDict.keys():
            refList.append(ref + ' ' + addDescDict[no])
        elif no in removeDescDict.keys():
            refList.append(removeDescDict[no])
        else:
            refList.append(ref)

    df['Reference 2'] = refList
      
    excelFile = outputFolder + '/' + bsfile
    print(excelFile)
    workbook = xlsxwriter.Workbook(excelFile)
    worksheet = workbook.add_worksheet() 
    workbook.close()

    
    #write pandas 
    writer = pd.ExcelWriter(excelFile, engine='openpyxl')
    writer.book = load_workbook(excelFile)
    # copy existing sheets
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    dfSummary.to_excel(writer, sheet_name='Sheet1', header = True, index = False)
    df[["Trn. Date", "Reference 1", "Reference 2", 'Debit Amount', 'Credit Amount', 'Balance']]\
    .to_excel(writer, sheet_name='Sheet1', startrow = dfSummary.shape[0] + 2, header = True, index = False)
    writer.save()

Dec2020_ManipulationResult/01_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/02_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/03_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/04_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/05_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/06_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/07_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/08_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/09_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/10_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/11_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/12_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/13_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/14_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/15_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/16_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/17_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/18_12 3207971832_2020.xlsx
Dec2020_ManipulationResult/1

In [ ]:
6/11, page 1799 and 1800